# Deep Learning

## Importing required libraries

In [ ]:
import os
os.environ['COMET_KERAS_HISTOGRAM_ACTIVATION_INDEX_LIST'] = "1,2"

from comet_ml import Experiment, ExistingExperiment
import tensorflow as tf
from tensorflow import keras
import numpy as np

## Loading Dataset

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = \
    fashion_mnist.load_data()

# Names of the integer classes, i.e., 0 -> T-short/top, 1 -> Trouser, etc.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Plotting some samples

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(test_images[4])
plt.axis('off')
plt.savefig('shoes.png')
plt.show()

In [ ]:
n_row = 5
n_col = 5
_, axs = plt.subplots(n_row, n_col, figsize=(12, 12))
axs = axs.flatten()
for img, ax in zip(train_images, axs):
    ax.imshow(img)
    ax.axis('off')
plt.savefig('examples.png')
plt.show()

## Building the Gradio interface

In [ ]:
import gradio as gr

model = None
def predict(image):
  image = image.reshape(-1, 28, 28, 1)
  prediction = model.predict(image).flatten()
  return {class_names[i]: float(prediction[i]) for i in range(len(class_names))}

In [ ]:
# initializing the input component
image = gr.inputs.Image(shape=(28, 28)) 
# initializing the output component 
label = gr.outputs.Label()
io = gr.Interface(fn = predict,inputs = image,outputs = label, title="Image Classification")
io.launch(inline=False)


## Creating Comet Experiment

In [ ]:
#experiment = Experiment(api_key='eLh0cH1J7CmpKR7x153LqCTZN',project_name='deep-learning', workspace='packt',auto_histogram_weight_logging=True,auto_histogram_gradient_logging=True,auto_histogram_activation_logging=True)
experiment = Experiment()

In [ ]:
experiment_key = experiment.get_key()

In [ ]:
io.integrate(comet_ml=experiment)

In [ ]:
experiment = ExistingExperiment(previous_experiment=experiment_key)

## Save some images

In [ ]:
# Reshape images for log api
train_img = np.reshape(train_images[0:5], (-1, 28, 28, 1))
test_img = np.reshape(test_images[0:5], (-1, 28, 28, 1))

@tf.function
def trace_model(x):
    return model(x)

LOG_DIR = 'logs'
file_writer = tf.summary.create_file_writer(LOG_DIR)
with file_writer.as_default():
  tf.summary.image("Training data", train_img, step=0)
  tf.summary.image("Test data", test_img, step=0)


## Building and fitting the model

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(
    train_images,
    train_labels,
    epochs=8,
    validation_data=(test_images, test_labels),
)

In [ ]:
print(model.summary())

## Plotting the model

In [ ]:
tf.keras.utils.plot_model(model, expand_nested=True)

## Logging confusion matrix

In [ ]:
preds = model.predict(test_images)
experiment.log_confusion_matrix(test_labels, preds)    

In [ ]:
experiment.end()